In [25]:
import pandas as pd
import numpy as np
import xarray as xr
import os, sys
import glob
import pdb
%%capture
from tqdm.notebook import tqdm, tnrange
tqdm().pandas()

UsageError: Line magic function `%%capture` not found.


Join multiple files into one using NCO utilities.
[here](http://forum.marine.copernicus.eu/discussion/205/how-to-mergeconcatenate-cmems-netcdf-files-tips/p1)

The following routine concats files without the need for NCO, but can be slow if there are many files.

In [2]:
files = glob.glob('/storage/ShenResults/*.csv')

In [18]:
def save_as_netcdf(ds, filename):
    filename = filename.replace('.csv', '.nc')
    comp = dict(zlib=True, complevel=9) # compresss level set to max
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(filename, encoding=encoding)
    
# convert csv to nc. Need to leave out index for now.
for file in tqdm(files): # each file is 
    try:
        df = pd.read_csv(file, index_col=0)
        df = df.drop_duplicates(keep='first')
        ds = ds.set_coords(['lat', 'long', 'tdx', 'pres'])
        ds = xr.Dataset.from_dataframe(df)
        save_as_netcdf(ds, file)
    except Exception as err:
        pdb.set_trace()
        print(err)
        continue

/home/tyler/anaconda3/envs/argo/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [32]:
dataSets = []
nfiles = glob.glob('/storage/ShenResults/*.nc')
for file in tqdm(nfiles):
    ds = xr.load_dataset(file)
    dataSets.append(ds)

In [ ]:
combined = xr.concat(dataSets, dim='pres') # can concat along pressure dimension

In [ ]:
save_as_netcdf(combined, 'meanTemp.nc')

In [6]:
file

'/storage/ShenResults/mTempData_3.75.csv'

In [6]:
comp = dict(zlib=True, complevel=9) # compresss level set to max
encoding = {var: comp for var in combined.data_vars}
combined.to_netcdf('mTempData.nc', encoding=encoding)

# Check if file was created properly

In [4]:
ds = xr.open_dataset('mTempData.nc')

In [5]:
ds

<xarray.Dataset>
Dimensions:    (lat: 552, long: 1233, pres: 26, tdx: 2)
Coordinates:
  * lat        (lat) float64 -89.0 -74.25 -73.75 -73.5 ... 72.0 74.5 75.25 78.25
  * long       (long) float64 -179.5 -179.2 -179.0 -178.8 ... 179.2 179.5 179.8
  * tdx        (tdx) int64 1093 1094
  * pres       (pres) float64 3.75 9.75 16.0 ... 1.45e+03 1.625e+03 1.875e+03
Data variables:
    mean       (lat, long, tdx, pres) float64 ...
    std        (lat, long, tdx, pres) float64 ...
    dof        (lat, long, tdx, pres) float64 ...
    startDate  (lat, long, tdx, pres) object ...
    endDate    (lat, long, tdx, pres) object ...

In [21]:
len(ds.lat.values)

552

In [22]:
len(ds.long.values)

1233

In [7]:
df = ds.to_dataframe()

KeyboardInterrupt: 

In [6]:
df = df.dropna(axis=0)

NameError: name 'df' is not defined

In [43]:
df.head()

mean       std   dof   startDate     endDate
lat   long pres  tdx                                                   
-89.0 -0.0 3.75  1094 -1.471143  0.004502  14.0  2018-12-27  2018-12-31
           2.25  1094 -1.470500  0.003775   8.0  2018-12-27  2018-12-31
           10.00 1094  0.823216  1.202438  37.0  2018-12-27  2018-12-31
           4.00  1094 -1.277733  0.158675  15.0  2018-12-27  2018-12-31
           5.00  1094 -0.959263  0.280545  19.0  2018-12-27  2018-12-31